# Assignment 5 - (Apache Kafka)


1. Setting up a Kafka Producer:
   a) Write a Python program to create a Kafka producer.
   b) Configure the producer to connect to a Kafka cluster.
   c) Implement logic to send messages to a Kafka topic.


In [ ]:
pip install confluent-kafka

In [ ]:
from confluent_kafka import Producer

def delivery_report(err, msg):
    """Callback function to handle the delivery report from the Kafka broker."""
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}]")

def create_kafka_producer(bootstrap_servers):
    """Create a Kafka producer with the specified bootstrap servers."""
    config = {
        "bootstrap.servers": bootstrap_servers,
        # Additional configuration options can be set here
    }

    return Producer(config)

def send_message(producer, topic, message):
    """Send a message to the specified Kafka topic."""
    producer.produce(topic, message.encode('utf-8'), callback=delivery_report)
    producer.flush()  # Wait for the message to be delivered

# Configuration
bootstrap_servers = "kafka1:9092,kafka2:9092,kafka3:9092"  # Comma-separated list of Kafka brokers
topic = "my-topic"  # Kafka topic to send messages to

# Create Kafka producer
producer = create_kafka_producer(bootstrap_servers)

# Send messages
message1 = "Hello, Kafka!"
message2 = "Welcome to Kafka producer."
send_message(producer, topic, message1)
send_message(producer, topic, message2)

# Clean up
producer.flush()
producer.close()


2. Setting up a Kafka Consumer:
   a) Write a Python program to create a Kafka consumer.
   b) Configure the consumer to connect to a Kafka cluster.
   c) Implement logic to consume messages from a Kafka topic.


In [ ]:
from confluent_kafka import Consumer, KafkaError

def consume_messages(consumer, topic):
    """Consume messages from the specified Kafka topic."""
    consumer.subscribe([topic])

    while True:
        msg = consumer.poll(1.0)  # Wait for messages for up to 1 second

        if msg is None:
            continue

        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition, continue polling
                continue
            else:
                print(f"Consumer error: {msg.error()}")
                break

        print(f"Received message: {msg.value().decode('utf-8')}")

    consumer.close()

# Configuration
bootstrap_servers = "kafka1:9092,kafka2:9092,kafka3:9092"  # Comma-separated list of Kafka brokers
topic = "my-topic"  # Kafka topic to consume messages from

# Create Kafka consumer
consumer = Consumer({
    'bootstrap.servers': bootstrap_servers,
    'group.id': 'my-consumer-group',  # Specify a consumer group ID
    'auto.offset.reset': 'earliest',  # Start consuming from the beginning of the topic
})

# Consume messages
consume_messages(consumer, topic)


3. Creating and Managing Kafka Topics:
   a) Write a Python program to create a new Kafka topic.
   b) Implement functionality to list existing topics.
   c) Develop logic to delete an existing Kafka topic.


In [ ]:
from confluent_kafka.admin import AdminClient, NewTopic

def create_topic(admin_client, topic_name, num_partitions, replication_factor):
    """Create a new Kafka topic."""
    topic = NewTopic(topic_name, num_partitions, replication_factor)
    admin_client.create_topics([topic])
    print(f"Topic '{topic_name}' created successfully.")

def list_topics(admin_client):
    """List all existing Kafka topics."""
    topics = admin_client.list_topics().topics
    print("Existing topics:")
    for topic_name in topics:
        print(topic_name)

def delete_topic(admin_client, topic_name):
    """Delete an existing Kafka topic."""
    admin_client.delete_topics([topic_name])
    print(f"Topic '{topic_name}' deleted successfully.")

# Configuration
bootstrap_servers = "kafka1:9092,kafka2:9092,kafka3:9092"  # Comma-separated list of Kafka brokers

# Create AdminClient
admin_client = AdminClient({'bootstrap.servers': bootstrap_servers})

# Create a new topic
create_topic(admin_client, "my-topic", 3, 1)

# List existing topics
list_topics(admin_client)

# Delete a topic
delete_topic(admin_client, "my-topic")


4. Producing and Consuming Messages:
   a) Write a Python program to produce messages to a Kafka topic.
   b) Implement logic to consume messages from the same Kafka topic.
   c) Test the end-to-end flow of message production and consumption.



In [ ]:
from confluent_kafka import Producer, Consumer, KafkaError

def delivery_report(err, msg):
    """Callback function to handle the delivery report from the Kafka broker."""
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}]")

def produce_messages(producer, topic, messages):
    """Produce messages to the specified Kafka topic."""
    for message in messages:
        producer.produce(topic, message.encode('utf-8'), callback=delivery_report)
        producer.poll(0.5)  # Poll for delivery reports every 0.5 seconds

    producer.flush()  # Wait for all messages to be delivered

def consume_messages(consumer, topic):
    """Consume messages from the specified Kafka topic."""
    consumer.subscribe([topic])

    while True:
        msg = consumer.poll(1.0)  # Wait for messages for up to 1 second

        if msg is None:
            continue

        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition, continue polling
                continue
            else:
                print(f"Consumer error: {msg.error()}")
                break

        print(f"Received message: {msg.value().decode('utf-8')}")

    consumer.close()

# Configuration
bootstrap_servers = "kafka1:9092,kafka2:9092,kafka3:9092"  # Comma-separated list of Kafka brokers
topic = "my-topic"  # Kafka topic to produce and consume messages

# Create Kafka producer
producer = Producer({
    "bootstrap.servers": bootstrap_servers,
    # Additional configuration options can be set here
})

# Create Kafka consumer
consumer = Consumer({
    "bootstrap.servers": bootstrap_servers,
    "group.id": "my-consumer-group",  # Specify a consumer group ID
    "auto.offset.reset": "earliest",  # Start consuming from the beginning of the topic
})

# Produce messages
messages_to_produce = ["Hello, Kafka!", "Welcome to message production."]
produce_messages(producer, topic, messages_to_produce)

# Consume messages
consume_messages(consumer, topic)



5. Working with Kafka Consumer Groups:
   a) Write a Python program to create a Kafka consumer within a consumer group.
   b) Implement logic to handle messages consumed by different consumers within the same group.
   c) Observe the behavior of consumer group rebalancing when adding or removing consumers.


In [ ]:
from confluent_kafka import Consumer, KafkaError

def consume_messages(consumer, topic):
    """Consume messages from the specified Kafka topic within a consumer group."""
    consumer.subscribe([topic])

    while True:
        msg = consumer.poll(1.0)  # Wait for messages for up to 1 second

        if msg is None:
            continue

        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition, continue polling
                continue
            else:
                print(f"Consumer error: {msg.error()}")
                break

        print(f"Consumer: {consumer.group_id()}, "
              f"Partition: {msg.partition()}, "
              f"Offset: {msg.offset()}, "
              f"Message: {msg.value().decode('utf-8')}")

    consumer.close()

# Configuration
bootstrap_servers = "kafka1:9092,kafka2:9092,kafka3:9092"  # Comma-separated list of Kafka brokers
topic = "my-topic"  # Kafka topic to consume messages from
group_id = "my-consumer-group"  # Consumer group ID

# Create Kafka consumer within the consumer group
consumer = Consumer({
    "bootstrap.servers": bootstrap_servers,
    "group.id": group_id,
    "auto.offset.reset": "earliest",  # Start consuming from the beginning of the topic
})

# Consume messages within the consumer group
consume_messages(consumer, topic)
